In [ ]:
# Returns back p(z|x) - probability sensor measures clean or dirty given state
# x (which could also be clean or dirty)
def sensor_model(z, x):
    if x == 'clean':
        if z == 'clean':
            return 0.9
        else: # z=dirty
            return 0.1
    else: # x=dirty
        if z == 'clean':
            return 0.3
        else: # z=dirty
            return 0.7

# Print out the possible values
for x in ['dirty', 'clean']:
    total_prob = 0.0
    for z in ['dirty', 'clean']:
        total_prob += sensor_model(z, x)
        print(f"p(z={z}|x={x}) = {sensor_model(z, x)}")
    assert(total_prob == 1.0)

In [ ]:
# Returns back p(xt_1|ut_1, xt_0) - probability that state xt_1 is now 'clean' or 'dirty'
# given that the prior state xt_0 was 'clean' or 'dirty' and the current
# action ut_1 is to 'vacuum' or 'donothing'
def motion_model(xt_1, ut_1, xt_0):
    if ut_1 == 'vacuum':
        if xt_0 == 'clean':
            if xt_1 == 'clean':
                return 1.0
            else: # xt_1=dirty
                return 0.0
        else: # xt_0=dirty
            if xt_1 == 'clean':
                return 0.7
            else: # xt_1=dirty
                return 0.3
    else: # ut_1='donothing'
        # State stays the same with absolute certainty
        if xt_1 == xt_0:
            return 1.0
        else:
            return 0.0

# Print out the possible values
for xt_0 in ['dirty', 'clean']:
    for ut_1 in ['vacuum', 'donothing']: 
        total_prob = 0.0
        for xt_1 in ['dirty', 'clean']:
            total_prob += motion_model(xt_1, ut_1, xt_0)
            print(f"p(xt_1={xt_1}|ut_1={ut_1}, xt_0={xt_0}) = {motion_model(xt_1, ut_1, xt_0)}")
        assert(total_prob == 1.0)

In [ ]:
# Initial belief of x - prob of both should add to 1.0
bel = [
    {'state': 'clean', 'prob': 0.5},
    {'state': 'dirty', 'prob': 0.5}
]

assert(bel[0]['prob'] + bel[1]['prob'] == 1.0)

In [ ]:
perceptual_or_action_list = [
    {'type': 'action', 'value': 'vacuum'},
    {'type': 'perceptual', 'value': 'clean'},
]

def bayes_filter(bel, d):
    new_bel = []
    # The 'correction' step
    if d['type'] == 'perceptual':
        eta = 0
        for b in bel:
            new_b = {
                'state': b['state'],
                'prob': sensor_model(d['value'], b['state']) * b['prob']
            }
            eta = eta + new_b['prob']
            new_bel.append(new_b)
        for new_b in new_bel:
            new_b['prob'] = (1.0/eta) * new_b['prob']
    # The 'prediction' step
    else: # d['type'] == 'action'
        for b_new in bel:
            prob = 0.0
            for b_prior in bel:
                prob = prob + (
                    motion_model(b_new['state'], d['value'], b_prior['state']) *
                    b_prior['prob'])
            new_b = {
                'state': b_new['state'],
                'prob': prob
            }
            new_bel.append(new_b)
    return new_bel

new_bel = bel
for perceptual_or_action in perceptual_or_action_list:
    new_bel = bayes_filter(new_bel, perceptual_or_action)

for b in new_bel:
    print(f"Belief that the current state is '{b['state']}' = {b['prob']}")